# API

In [ ]:
import random
from calendar import monthrange
from datetime import date, timedelta
from time import sleep

from fastapi.testclient import TestClient
from httpx import Response

from voxrow.web.domain.value_objects import Settings
from voxrow.web.entrypoint import app

In [ ]:
settings: Settings = Settings()
client: TestClient = TestClient(app)

## /idx/stock-summary

In [ ]:
# Oldest date is 2020-01-02
year: int = 2025
month: int = 10
start_date: date = date(year, month, 1)
days: int = monthrange(year, month)[1]

for day in range(days):
    current_date: date = start_date + timedelta(days=day)
    resp: Response = client.get(
        "/idx/stock-summary",
        headers=dict(Authorization=f"Bearer {settings.cron_secret.get_secret_value()}"),
        params=dict(date=str(current_date)),
        timeout=60 * 1.5,
    )

    print(f"{current_date} | status_code: {resp.status_code}")
    sleep(random.uniform(3.0, 5.0))

# Duckdb

## Preparation

In [ ]:
from os import environ

from voxrow.web.domain.value_objects import Settings

In [ ]:
settings: Settings = Settings()
environ["AWS_ACCESS_KEY_ID"] = settings.cloudflare_r2.aws_access_key_id.get_secret_value()
environ["AWS_SECRET_ACCESS_KEY"] = settings.cloudflare_r2.aws_secret_access_key.get_secret_value()

In [ ]:
%load_ext magic_duckdb
%dql -t relation
%dql SELECT 'Hanya Tes' ini;

In [ ]:
%%dql -j
CREATE OR REPLACE
SECRET (
    TYPE r2,
    PROVIDER credential_chain,
    CHAIN env,
    ENDPOINT '{{ settings.cloudflare_r2.endpoint_url.host }}'
);

## Queries

In [ ]:
%%dql
SELECT
    *
FROM
    'r2://datalake/idx.co.id/GetStockSummary/*.json.gz'
;